In [1]:
import pandas as pd
import matplotlib.pyplot as plt 
import numpy as np
import seaborn as sns
sns.set()
from sklearn.model_selection import StratifiedKFold, cross_val_score, RandomizedSearchCV

In [2]:
import catboost as cb
import xgboost as xgb
from sklearn.linear_model import SGDClassifier

In [3]:
train = pd.read_csv('X_train.csv')
test = pd.read_csv('X_test.csv')

In [4]:
train = train.drop(['Unnamed: 0'], axis=1)
test = test.drop(['Unnamed: 0'], axis=1)

In [5]:
target = train['quality']
train = train.drop(['quality'], axis=1)

## Data clening

In [6]:
mapping = {3:0, 4:0, 5:0, 6:0, 7:1, 8:1}
target = target.map(mapping)

In [7]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler
scaler = MinMaxScaler()

train = scaler.fit_transform(train)
test = scaler.fit_transform(test)

In [8]:
k_fold = StratifiedKFold(n_splits=5, shuffle=True)

### polyfeatures

In [9]:
from sklearn.preprocessing import PolynomialFeatures

In [10]:
# poly = PolynomialFeatures(2)
# train_poly = poly.fit_transform(train)
# test_poly = poly.fit_transform(test)

### Modeling

In [11]:
from sklearn.model_selection import cross_val_score
from sklearn.datasets import make_blobs
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.tree import DecisionTreeClassifier

In [12]:
from sklearn.ensemble import RandomForestClassifier, VotingClassifier

In [13]:
sgd_clf = SGDClassifier(loss="hinge", penalty="l2", max_iter=5)

In [14]:
cat_clf = cb.CatBoostClassifier()

In [15]:
xgb_clf = xgb.XGBClassifier()

In [16]:
clf_1 = DecisionTreeClassifier(max_depth=None, min_samples_split=2,
    random_state=0)
                   
clf_2 = RandomForestClassifier(n_estimators=10, max_depth=None,
    min_samples_split=2, random_state=0)
                           
clf_3 = ExtraTreesClassifier(n_estimators=10, max_depth=None,
    min_samples_split=2, random_state=0)

In [17]:
eclf1 = VotingClassifier(estimators=[
        ('lr', clf_1), ('rf', clf_2), ('gnb', clf_3)], voting='soft')

In [18]:
%time
cat_clf_cv = cross_val_score(eclf1, train, target, cv = k_fold, scoring='f1', n_jobs=8)

CPU times: user 3 µs, sys: 1e+03 ns, total: 4 µs
Wall time: 8.82 µs


/home/ubuntu/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/ubuntu/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/ubuntu/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/ubuntu/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is a

In [19]:
cat_clf_cv.mean() * 10**5

55207.88379347415

|попытка|скор|
|---|---|
|1.|56053 - 1000|
|2.min-max|57693|

In [20]:
from sklearn.model_selection import GridSearchCV

In [21]:
np.arange(0.0, 1.0, 0.1)

array([0. , 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9])

In [24]:
params={
    'max_depth': list(range(0,10,1)), # 5 is good but takes too long in kaggle env
    'subsample': np.arange(0.0, 1.0, 0.1),
    'colsample_bytree': np.arange(0.0, 1.0, 0.1),
    'n_estimators': list(range(80,2000,100)),
    'reg_alpha': np.arange(0.0, 1.0, 0.1)
}

xgb_clf = xgb.XGBClassifier()
rs = RandomizedSearchCV(xgb_clf,
                  params,
                  cv=5,
                  n_jobs=8,
                  verbose=10,
                 return_train_score=True)

In [25]:
%time
rs.fit(train, target)

CPU times: user 4 µs, sys: 1 µs, total: 5 µs
Wall time: 11.4 µs
Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV] subsample=0.7000000000000001, reg_alpha=0.9, n_estimators=1080, max_depth=9, colsample_bytree=0.1 
[CV] subsample=0.7000000000000001, reg_alpha=0.9, n_estimators=1080, max_depth=9, colsample_bytree=0.1 
[CV] subsample=0.7000000000000001, reg_alpha=0.9, n_estimators=1080, max_depth=9, colsample_bytree=0.1 
[CV] subsample=0.7000000000000001, reg_alpha=0.9, n_estimators=1080, max_depth=9, colsample_bytree=0.1 
[CV] subsample=0.7000000000000001, reg_alpha=0.9, n_estimators=1080, max_depth=9, colsample_bytree=0.1 
[CV] subsample=0.0, reg_alpha=0.30000000000000004, n_estimators=1880, max_depth=2, colsample_bytree=0.1 
[CV] subsample=0.0, reg_alpha=0.30000000000000004, n_estimators=1880, max_depth=2, colsample_bytree=0.1 
[CV] subsample=0.0, reg_alpha=0.30000000000000004, n_estimators=1880, max_depth=2, colsample_bytree=0.1 


/home/ubuntu/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/ubuntu/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


[CV]  subsample=0.0, reg_alpha=0.30000000000000004, n_estimators=1880, max_depth=2, colsample_bytree=0.1, score=0.8671875, total=   0.4s
[CV] subsample=0.0, reg_alpha=0.30000000000000004, n_estimators=1880, max_depth=2, colsample_bytree=0.1 


/home/ubuntu/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/ubuntu/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/ubuntu/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


[CV]  subsample=0.0, reg_alpha=0.30000000000000004, n_estimators=1880, max_depth=2, colsample_bytree=0.1, score=0.8671875, total=   0.4s
[CV] subsample=0.0, reg_alpha=0.30000000000000004, n_estimators=1880, max_depth=2, colsample_bytree=0.1 


...

/home/ubuntu/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/ubuntu/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


[CV]  subsample=0.7000000000000001, reg_alpha=0.5, n_estimators=580, max_depth=2, colsample_bytree=0.5, score=0.87109375, total=   0.3s


/home/ubuntu/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/ubuntu/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/ubuntu/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


[CV]  subsample=0.9, reg_alpha=0.1, n_estimators=80, max_depth=4, colsample_bytree=0.1, score=0.8671875, total=   0.0s


JoblibXGBoostError: JoblibXGBoostError
___________________________________________________________________________
Multiprocessing exception:
...........................................................................
/home/ubuntu/anaconda3/lib/python3.6/runpy.py in _run_module_as_main(mod_name='ipykernel_launcher', alter_argv=1)
    188         sys.exit(msg)
    189     main_globals = sys.modules["__main__"].__dict__
    190     if alter_argv:
    191         sys.argv[0] = mod_spec.origin
    192     return _run_code(code, main_globals, None,
--> 193                      "__main__", mod_spec)
        mod_spec = ModuleSpec(name='ipykernel_launcher', loader=<_f...b/python3.6/site-packages/ipykernel_launcher.py')
    194 
    195 def run_module(mod_name, init_globals=None,
    196                run_name=None, alter_sys=False):
    197     """Execute a module's code without importing it

...........................................................................
/home/ubuntu/anaconda3/li

In [27]:
rs.score

<bound method BaseSearchCV.score of RandomizedSearchCV(cv=5, error_score='raise',
          estimator=XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1),
          fit_params=None, iid=True, n_iter=10, n_jobs=8,
          param_distributions={'max_depth': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9], 'subsample': array([0. , 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]), 'colsample_bytree': array([0. , 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]), 'n_estimators': [80, 180, 280, 380, 480, 580, 680, 780, 880, 980, 1080, 1180, 1280, 1380, 1480, 1580, 1680, 1780, 1880, 1980], 'reg_alpha': array([0. , 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9])},
          pre_dispatch='

In [28]:
xgb_clf = xgb.XGBClassifier(colsample_bytree=0.5, max_depth=4, n_estimators=200, reg_alpha=0.04, subsample=0.6)

## Subbmit

In [29]:
eclf1.fit(train,target)

VotingClassifier(estimators=[('lr', DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf..._estimators=10, n_jobs=1,
           oob_score=False, random_state=0, verbose=0, warm_start=False))],
         flatten_transform=None, n_jobs=1, voting='soft', weights=None)

In [30]:
ans = eclf1.predict(test)

/home/ubuntu/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [31]:
ans = pd.Series(ans)

In [32]:
import time

In [33]:
def make_submission(predicted):
    submission = []
    for i in predicted:
        submission.append(int(i))
        
    f = open('submission{}.txt'.format('::'.join(time.ctime().split()[2:4])), 'w')
    for i in submission:
        f.write(str(i) + '\r')

In [34]:
make_submission(ans)